In [31]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from urllib.parse import quote

In [17]:
df = pd.read_csv("MultiLabel.csv")
df.head()

,artist,genre,title,album,year,lyrics,labels
0,Nirvana,Rock,You Know You’re Right,Nirvana,2002.0,I will never bother you\nI will never promise ...,"Calmness, Sadness"
1,Damian Marley,Reggae,Here We Go,Stony Hill,2017.0,Here we go\nMy big ego is gonna get me in trou...,"Power, Tension"
2,The Mission UK,Rock,Jade,Another Fall from Grace,2016.0,She came as Lolita dressed as Venus\nAnd adorn...,"Amazement, Calmness, Solemnity, Tenderness"
3,UB40,Reggae,Food For Thought,Signing Off,1980.0,"Ivory Madonna, dying in the dust\nWaiting for ...","Joyful activation, Sadness, Tension"
4,Johnny Cash,Country,I’ve Been Everywhere,American II: Unchained,1996.0,I was totin' my pack along the dusty Winnemucc...,"Amazement, Calmness, Joyful activation"


In [18]:
labels = df["labels"].unique()
labels_dict = set()
for label in labels:
    label = label.split(',')
    for lab in label:
        labels_dict.add(lab)
print(labels_dict)
#'01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
#    '05': 'angry', '06': 'fearful', '07': 'disgust', '08': 'surprised'
#}


{'Solemnity', 'Tenderness', 'Tension', 'Power', ' Joyful activation', 'Amazement', 'Nostalgia', 'Joyful activation', 'Calmness', ' Solemnity', ' Tenderness', ' Tension', ' Nostalgia', ' Sadness', ' Power', ' Calmness', 'Sadness'}


In [20]:
def replace_emotions(emotions):
    emotions = emotions.replace('Calmness', 'calm')
    emotions = emotions.replace('Solemnity', 'sad')
    emotions = emotions.replace('Tenderness', 'calm')
    emotions = emotions.replace('Power', 'happy')
    emotions = emotions.replace('Joyful activation', 'happy')
    emotions = emotions.replace('Amazement', 'happy')
    emotions = emotions.replace('Nostalgia', 'calm')
    emotions = emotions.replace('Joyful activation', 'happy')
    emotions = emotions.replace('Tension', 'fearful')
    emotions = emotions.replace('Sadness', 'sad')
    return emotions
df["labels"] = df["labels"].apply(replace_emotions)
df['labels'].unique()
df.head()

,artist,genre,title,album,year,lyrics,labels
0,Nirvana,Rock,You Know You’re Right,Nirvana,2002.0,I will never bother you\nI will never promise ...,"calm, sad"
1,Damian Marley,Reggae,Here We Go,Stony Hill,2017.0,Here we go\nMy big ego is gonna get me in trou...,"happy, fearful"
2,The Mission UK,Rock,Jade,Another Fall from Grace,2016.0,She came as Lolita dressed as Venus\nAnd adorn...,"happy, calm, sad, calm"
3,UB40,Reggae,Food For Thought,Signing Off,1980.0,"Ivory Madonna, dying in the dust\nWaiting for ...","happy, sad, fearful"
4,Johnny Cash,Country,I’ve Been Everywhere,American II: Unchained,1996.0,I was totin' my pack along the dusty Winnemucc...,"happy, calm, happy"


In [21]:
df['labels'].unique()

array(['calm, sad', 'happy, fearful', 'happy, calm, sad, calm',
       'happy, sad, fearful', 'happy, calm, happy', 'sad, sad, fearful',
       'happy, happy, sad, calm', 'calm, happy, sad, calm',
       'happy, happy, calm', 'sad, calm, fearful',
       'calm, calm, happy, sad, sad', 'calm, sad, fearful',
       'calm, sad, calm', 'happy, happy, sad, fearful',
       'happy, sad, sad, fearful', 'happy, happy, sad', 'sad, calm',
       'calm, sad, calm, fearful', 'calm, happy, calm, calm',
       'calm, calm', 'calm, happy, sad, sad', 'calm, sad, sad, fearful',
       'happy, sad, calm', 'calm, happy, sad, sad, fearful', 'happy, sad',
       'happy, sad, sad', 'calm, happy, fearful', 'happy, calm',
       'happy, happy, calm, sad, sad, fearful',
       'calm, happy, sad, fearful', 'happy, happy, calm, sad, calm',
       'calm, happy, sad', 'calm', 'happy, happy, calm, happy, sad',
       'calm, calm, fearful', 'calm, happy, calm',
       'happy, happy, fearful', 'happy, calm, happy, ha

In [37]:


load_dotenv()
spotify_key = os.getenv("SPOTIFY_API_KEY")
def search_spotify(artist, title):
    query = f"artist:{artist} track:{title}"
    encoded_query = quote(query)
    url = f"https://api.spotify.com/v1/search?q={encoded_query}&type=track&limit=1"
    headers = {
        "Authorization": f"Bearer {spotify_key}"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if 'tracks' in data and data['tracks']['items']:
            return data['tracks']['items'][0]['external_urls']['spotify']
    return None
df['spotify_url'] = df.apply(
    lambda row: search_spotify(row['artist'], row['title']),
    axis=1
)
df

,artist,genre,title,album,year,lyrics,labels,spotify_url
0,Nirvana,Rock,You Know You’re Right,Nirvana,2002.0,I will never bother you\nI will never promise ...,"calm, sad",https://open.spotify.com/track/5O8L4I2S4izFGDj...
1,Damian Marley,Reggae,Here We Go,Stony Hill,2017.0,Here we go\nMy big ego is gonna get me in trou...,"happy, fearful",https://open.spotify.com/track/0dmYihdEAM4MvsB...
2,The Mission UK,Rock,Jade,Another Fall from Grace,2016.0,She came as Lolita dressed as Venus\nAnd adorn...,"happy, calm, sad, calm",None
3,UB40,Reggae,Food For Thought,Signing Off,1980.0,"Ivory Madonna, dying in the dust\nWaiting for ...","happy, sad, fearful",https://open.spotify.com/track/5YkkeIiErNf2ckv...
4,Johnny Cash,Country,I’ve Been Everywhere,American II: Unchained,1996.0,I was totin' my pack along the dusty Winnemucc...,"happy, calm, happy",https://open.spotify.com/track/2vLPGv0TAzbY68z...
...,...,...,...,...,...,...,...,...
1155,The Beatles,Rock,Yesterday,Help!,1965.0,Yesterday\nAll my troubles seemed so far away\...,"calm, sad, calm",None
1156,The Sundays,Rock,Love,Blind,1992.0,Picture myself as a thin white child\nBack to ...,"calm, happy, calm",None
1157,The Cure,Rock,Killing an Arab,Boys Don’t Cry,1978.0,Standing on the beach\nWith a gun in my hand\n...,"happy, sad, fearful",None
1158,Garth Brooks,Country,Mom,Man Against Machine,2014.0,"A little baby told God\nHey I'm kinda scared, ...",calm,None


In [46]:
df[df["spotify_url"].isna()]
len(df[df["spotify_url"].isna()])/len(df)

0.44396551724137934

In [47]:
df = df[df['spotify_url'].notna()]
len(df[df["spotify_url"].isna()])/len(df)

0.0

In [49]:
print(df.shape)
df.head()

(645, 8)


,artist,genre,title,album,year,lyrics,labels,spotify_url
0,Nirvana,Rock,You Know You’re Right,Nirvana,2002.0,I will never bother you\nI will never promise ...,"calm, sad",https://open.spotify.com/track/5O8L4I2S4izFGDj...
1,Damian Marley,Reggae,Here We Go,Stony Hill,2017.0,Here we go\nMy big ego is gonna get me in trou...,"happy, fearful",https://open.spotify.com/track/0dmYihdEAM4MvsB...
3,UB40,Reggae,Food For Thought,Signing Off,1980.0,"Ivory Madonna, dying in the dust\nWaiting for ...","happy, sad, fearful",https://open.spotify.com/track/5YkkeIiErNf2ckv...
4,Johnny Cash,Country,I’ve Been Everywhere,American II: Unchained,1996.0,I was totin' my pack along the dusty Winnemucc...,"happy, calm, happy",https://open.spotify.com/track/2vLPGv0TAzbY68z...
5,Alan Jackson,Country,It’s Five O’Clock Somewhere,Greatest Hits Volume II Disc 1,2003.0,The sun is hot and that old clock is movin' sl...,"sad, sad, fearful",https://open.spotify.com/track/7KysI43rsjlhjTe...


In [53]:
# df.drop(columns=["lyrics"], inplace=True)
# df
df.to_csv("songs.csv")